# Oil Spill Detection - Data Preprocessing Pipeline

This notebook implements a robust preprocessing pipeline for oil spill detection based on EDA findings.

## Key Requirements:
- **Same Image Size**: Consistent 256x256 resolution across train/val/test
- **Normalization Consistency**: Same 0-1 scaling for all datasets
- **Shuffling**: Train=True, Val/Test=False
- **Data Balance**: Only augment train set, keep val/test untouched
- **Batch Size**: Consistent across all sets
- **Visual Verification**: Check samples after preprocessing

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
from sklearn.model_selection import train_test_split
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Create results directory
os.makedirs('../results/preprocessing', exist_ok=True)
os.makedirs('../data/processed', exist_ok=True)

print("Libraries imported successfully")
print("Results directories created")

## 1. Preprocessing Configuration

In [ ]:
# Enhanced configuration with consistency requirements
CONFIG = {
    'target_size': (256, 256),  # CONSISTENT across all splits
    'batch_size': 32,  # SAME for train/val/test
    'normalize_range': [0, 1],  # CONSISTENT 0-1 scaling
    'binary_threshold': 127,
    'augmentation_prob': 0.5,
    'dataset_path': '../dataset',
    'processed_path': '../data/processed',
    'min_spill_area': 0.1,
    'balance_threshold': 0.3,
    # Shuffling configuration
    'shuffle_train': True,   # Train: shuffle for randomization
    'shuffle_val': False,    # Val: no shuffle for consistent evaluation
    'shuffle_test': False    # Test: no shuffle for consistent evaluation
}

print("Enhanced Preprocessing Configuration:")
print("=" * 50)
for key, value in CONFIG.items():
    print(f"  {key}: {value}")
print("=" * 50)
print("\n✅ Key Rules:")
print("  1. Same 256x256 size everywhere")
print("  2. Consistent 0-1 normalization")
print("  3. Train shuffle=True, Val/Test shuffle=False")
print("  4. Only augment train set")
print("  5. Same batch size (32) for all")
print("  6. Visual verification included")

## 2. Enhanced Mask Conversion Functions

In [ ]:
def convert_rgb_mask_to_binary(mask_path, threshold=None):
    """
    Convert RGB mask to binary format with enhanced error handling
    Args:
        mask_path: Path to RGB mask
        threshold: Threshold for binary conversion
    Returns:
        Binary mask (0=non-spill, 1=spill)
    """
    if threshold is None:
        threshold = CONFIG['binary_threshold']
        
    try:
        # Load mask - try different methods
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        
        if mask is None:
            # Try with PIL if cv2 fails
            mask_pil = Image.open(mask_path).convert('L')
            mask = np.array(mask_pil)
        
        if mask is None:
            return None, f"Could not load mask: {mask_path}"
        
        # Convert to binary (any pixel above threshold becomes 1)
        binary_mask = (mask > threshold).astype(np.uint8)
        
        # Calculate spill area percentage
        spill_area = (np.sum(binary_mask) / binary_mask.size) * 100
        
        return binary_mask, spill_area
        
    except Exception as e:
        return None, f"Error converting mask {mask_path}: {e}"

# Enhanced visualization with better error handling and statistics
def visualize_mask_conversion(image_path, mask_path, num_samples=3):
    """
    Visualize original vs converted masks with statistics
    """
    if not (os.path.exists(image_path) and os.path.exists(mask_path)):
        print(f"Paths do not exist: {image_path} or {mask_path}")
        return
    
    image_files = [f for f in os.listdir(image_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    if len(image_files) == 0:
        print("No image files found")
        return
    
    sample_files = np.random.choice(image_files, min(num_samples, len(image_files)), replace=False)
    
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, 5*num_samples))
    if num_samples == 1:
        axes = axes.reshape(1, -1)
    
    conversion_stats = {'successful': 0, 'failed': 0, 'spill_areas': []}
    
    for i, img_file in enumerate(sample_files):
        try:
            # Load original image
            img = cv2.imread(os.path.join(image_path, img_file))
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # Load original mask
            mask_file = img_file
            original_mask = cv2.imread(os.path.join(mask_path, mask_file), cv2.IMREAD_GRAYSCALE)
            
            # Convert mask
            binary_mask, spill_info = convert_rgb_mask_to_binary(os.path.join(mask_path, mask_file))
            
            if binary_mask is not None:
                conversion_stats['successful'] += 1
                conversion_stats['spill_areas'].append(spill_info)
            else:
                conversion_stats['failed'] += 1
                print(f"Failed to convert: {spill_info}")
            
            # Plot original image
            axes[i, 0].imshow(img_rgb)
            axes[i, 0].set_title(f'Original Image\n{img_file}\nSize: {img_rgb.shape}', fontsize=10)
            axes[i, 0].axis('off')
            
            # Plot original mask
            if original_mask is not None:
                axes[i, 1].imshow(original_mask, cmap='gray')
                axes[i, 1].set_title(f'Original Mask\nSize: {original_mask.shape}', fontsize=10)
            axes[i, 1].axis('off')
            
            # Plot binary mask
            if binary_mask is not None:
                axes[i, 2].imshow(binary_mask, cmap='gray')
                axes[i, 2].set_title(f'Binary Mask\nSpill Area: {spill_info:.1f}%', fontsize=10)
            axes[i, 2].axis('off')
            
        except Exception as e:
            print(f"Error processing {img_file}: {e}")
            conversion_stats['failed'] += 1
    
    plt.tight_layout()
    plt.savefig('../results/preprocessing/mask_conversion_examples.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Print conversion statistics
    print(f"\nMask Conversion Statistics:")
    print(f"  Successful conversions: {conversion_stats['successful']}")
    print(f"  Failed conversions: {conversion_stats['failed']}")
    if conversion_stats['spill_areas']:
        print(f"  Average spill area: {np.mean(conversion_stats['spill_areas']):.2f}%")
        print(f"  Spill area range: {np.min(conversion_stats['spill_areas']):.2f}% - {np.max(conversion_stats['spill_areas']):.2f}%")

# Test mask conversion
train_images_path = os.path.join(CONFIG['dataset_path'], 'train', 'images')
train_masks_path = os.path.join(CONFIG['dataset_path'], 'train', 'masks')

print("Testing mask conversion...")
visualize_mask_conversion(train_images_path, train_masks_path, 3)

## 3. Enhanced Image Preprocessing Functions

In [ ]:
# Enhanced preprocessing with strict consistency checks
def preprocess_image_consistent(image_path, target_size=None, normalize_range=None):
    """
    Preprocess image with STRICT consistency requirements
    """
    if target_size is None:
        target_size = CONFIG['target_size']
    if normalize_range is None:
        normalize_range = CONFIG['normalize_range']
    
    try:
        # Load image
        image = cv2.imread(image_path)
        if image is None:
            return None, "Could not load image"
        
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # CONSISTENT resize to exact target size
        image_resized = cv2.resize(image_rgb, target_size)
        
        # CONSISTENT normalization to [0, 1]
        image_normalized = image_resized.astype(np.float32) / 255.0
        
        # Verify consistency
        assert image_normalized.shape[:2] == target_size, f"Size mismatch: {image_normalized.shape[:2]} != {target_size}"
        assert image_normalized.min() >= 0 and image_normalized.max() <= 1, f"Normalization error: range [{image_normalized.min():.3f}, {image_normalized.max():.3f}]"
        
        return image_normalized, "success"
        
    except Exception as e:
        return None, f"Error: {e}"

def preprocess_mask_consistent(mask_path, target_size=None):
    """
    Preprocess mask with STRICT consistency requirements
    """
    if target_size is None:
        target_size = CONFIG['target_size']
    
    try:
        # Convert to binary
        binary_mask, spill_area = convert_rgb_mask_to_binary(mask_path)
        
        if binary_mask is None:
            return None, 0, f"Binary conversion failed: {spill_area}"
        
        # CONSISTENT resize to exact target size
        mask_resized = cv2.resize(binary_mask, target_size, interpolation=cv2.INTER_NEAREST)
        
        # Verify consistency
        assert mask_resized.shape == target_size, f"Mask size mismatch: {mask_resized.shape} != {target_size}"
        assert set(np.unique(mask_resized)).issubset({0, 1}), f"Mask values not binary: {np.unique(mask_resized)}"
        
        return mask_resized, spill_area, "success"
        
    except Exception as e:
        return None, 0, f"Error: {e}"

## 4. Enhanced Data Augmentation Pipeline

In [ ]:
# Enhanced augmentation with train-only policy
def get_augmentation_pipeline_enhanced(split='train', target_size=None):
    """
    Create augmentation pipeline with STRICT split-based rules
    Args:
        split: 'train', 'val', or 'test'
        target_size: Target image size
    """
    if target_size is None:
        target_size = CONFIG['target_size']
    
    if split == 'train':
        # ONLY train gets augmentation
        return A.Compose([
            A.Resize(target_size[0], target_size[1]),
            
            # Geometric transformations
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.3),
            A.Rotate(limit=15, p=0.5, border_mode=cv2.BORDER_CONSTANT, value=0),
            
            # Photometric transformations
            A.RandomBrightnessContrast(
                brightness_limit=0.2,
                contrast_limit=0.2,
                p=0.5
            ),
            A.HueSaturationValue(
                hue_shift_limit=10,
                sat_shift_limit=20,
                val_shift_limit=10,
                p=0.3
            ),
            
            # Noise and blur
            A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
            A.GaussianBlur(blur_limit=(3, 5), p=0.2),
            
            # Elastic transformations
            A.ElasticTransform(
                alpha=1, sigma=50, alpha_affine=50,
                border_mode=cv2.BORDER_CONSTANT, value=0, p=0.2
            ),
            
            # CONSISTENT normalization
            A.Normalize(mean=[0.0, 0.0, 0.0], std=[1.0, 1.0, 1.0])  # Keep 0-1 range
        ])
    else:
        # Val/Test: ONLY resize and normalize (NO augmentation)
        return A.Compose([
            A.Resize(target_size[0], target_size[1]),
            A.Normalize(mean=[0.0, 0.0, 0.0], std=[1.0, 1.0, 1.0])  # Keep 0-1 range
        ])

print("✅ Augmentation Rules:")
print("  - Train: Full augmentation pipeline")
print("  - Val/Test: Only resize + normalize (NO augmentation)")
print("  - All splits: Same target size and normalization")

In [ ]:
# Added visual verification function
def verify_preprocessing_samples(split='train', num_samples=6):
    """
    Visual verification of preprocessing pipeline
    """
    images_path = os.path.join(CONFIG['dataset_path'], split, 'images')
    masks_path = os.path.join(CONFIG['dataset_path'], split, 'masks')
    
    image_files = [f for f in os.listdir(images_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))][:num_samples]
    
    fig, axes = plt.subplots(num_samples, 4, figsize=(16, 4*num_samples))
    fig.suptitle(f'Preprocessing Verification - {split.upper()} Split', fontsize=16, fontweight='bold')
    
    augmentation_pipeline = get_augmentation_pipeline_enhanced(split)
    
    for i, img_file in enumerate(image_files):
        try:
            img_path = os.path.join(images_path, img_file)
            mask_path = os.path.join(masks_path, img_file)
            
            # Load original
            original_img = cv2.imread(img_path)
            original_img_rgb = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
            
            # Load original mask
            original_mask = cv2.imread(mask_path)
            original_mask_rgb = cv2.cvtColor(original_mask, cv2.COLOR_BGR2RGB)
            
            # Preprocess
            processed_img, img_status = preprocess_image_consistent(img_path)
            processed_mask, spill_area, mask_status = preprocess_mask_consistent(mask_path)
            
            if processed_img is not None and processed_mask is not None:
                # Apply augmentation if train split
                if split == 'train':
                    # Convert to uint8 for augmentation
                    img_uint8 = (processed_img * 255).astype(np.uint8)
                    augmented = augmentation_pipeline(image=img_uint8, mask=processed_mask)
                    final_img = augmented['image']
                    final_mask = augmented['mask']
                else:
                    # For val/test, just apply resize and normalize
                    img_uint8 = (processed_img * 255).astype(np.uint8)
                    transformed = augmentation_pipeline(image=img_uint8, mask=processed_mask)
                    final_img = transformed['image']
                    final_mask = transformed['mask']
                
                # Plot results
                axes[i, 0].imshow(original_img_rgb)
                axes[i, 0].set_title(f'Original\n{original_img_rgb.shape}')
                axes[i, 0].axis('off')
                
                axes[i, 1].imshow(original_mask_rgb)
                axes[i, 1].set_title(f'Original Mask\n{original_mask_rgb.shape}')
                axes[i, 1].axis('off')
                
                axes[i, 2].imshow(processed_img)
                axes[i, 2].set_title(f'Processed\n{processed_img.shape}\nRange: [{processed_img.min():.2f}, {processed_img.max():.2f}]')
                axes[i, 2].axis('off')
                
                axes[i, 3].imshow(final_mask, cmap='gray')
                axes[i, 3].set_title(f'Final Mask\n{final_mask.shape}\nSpill: {spill_area:.1f}%')
                axes[i, 3].axis('off')
                
            else:
                for j in range(4):
                    axes[i, j].text(0.5, 0.5, f'Error: {img_status}', 
                                  ha='center', va='center', transform=axes[i, j].transAxes)
                    axes[i, j].axis('off')
                    
        except Exception as e:
            for j in range(4):
                axes[i, j].text(0.5, 0.5, f'Error: {e}', 
                              ha='center', va='center', transform=axes[i, j].transAxes)
                axes[i, j].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n✅ Visual verification completed for {split} split")
    print(f"   - Target size: {CONFIG['target_size']}")
    print(f"   - Normalization: {CONFIG['normalize_range']}")
    print(f"   - Augmentation: {'Yes' if split == 'train' else 'No'}")

## 5. Dataset Balance Analysis and Processing

In [ ]:
# Enhanced dataset processing with consistency validation
def process_dataset_split_enhanced(split='train'):
    """
    Process dataset split with STRICT consistency validation
    """
    print(f"\n{'='*60}")
    print(f"PROCESSING {split.upper()} SPLIT WITH CONSISTENCY CHECKS")
    print(f"{'='*60}")
    
    images_path = os.path.join(CONFIG['dataset_path'], split, 'images')
    masks_path = os.path.join(CONFIG['dataset_path'], split, 'masks')
    
    if not (os.path.exists(images_path) and os.path.exists(masks_path)):
        print(f"❌ Paths for {split} split do not exist")
        return None
    
    image_files = [f for f in os.listdir(images_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    if not image_files:
        print(f"❌ No images found in {split} split")
        return None
    
    stats = {
        'split': split,
        'total_images': len(image_files),
        'successful_processing': 0,
        'failed_processing': 0,
        'spill_images': 0,
        'non_spill_images': 0,
        'consistency_checks': {
            'size_consistent': 0,
            'normalization_consistent': 0,
            'mask_binary': 0
        },
        'spill_areas': [],
        'processing_errors': []
    }
    
    print(f"Processing {len(image_files)} images...")
    
    for img_file in tqdm(image_files, desc=f"Processing {split}"):
        try:
            img_path = os.path.join(images_path, img_file)
            mask_path = os.path.join(masks_path, img_file)
            
            # Process with consistency checks
            processed_img, img_status = preprocess_image_consistent(img_path)
            processed_mask, spill_area, mask_status = preprocess_mask_consistent(mask_path)
            
            if processed_img is not None and processed_mask is not None:
                stats['successful_processing'] += 1
                
                # Consistency checks
                if processed_img.shape[:2] == CONFIG['target_size']:
                    stats['consistency_checks']['size_consistent'] += 1
                
                if 0 <= processed_img.min() and processed_img.max() <= 1:
                    stats['consistency_checks']['normalization_consistent'] += 1
                
                if set(np.unique(processed_mask)).issubset({0, 1}):
                    stats['consistency_checks']['mask_binary'] += 1
                
                # Spill classification
                if spill_area > CONFIG['min_spill_area']:
                    stats['spill_images'] += 1
                    stats['spill_areas'].append(spill_area)
                else:
                    stats['non_spill_images'] += 1
                    
            else:
                stats['failed_processing'] += 1
                stats['processing_errors'].append(f"{img_file}: {img_status}, {mask_status}")
                
        except Exception as e:
            stats['failed_processing'] += 1
            stats['processing_errors'].append(f"{img_file}: {e}")
    
    # Calculate final metrics
    total_processed = stats['successful_processing']
    if total_processed > 0:
        stats['spill_ratio'] = stats['spill_images'] / total_processed
        stats['consistency_rate'] = {
            'size': stats['consistency_checks']['size_consistent'] / total_processed,
            'normalization': stats['consistency_checks']['normalization_consistent'] / total_processed,
            'mask_binary': stats['consistency_checks']['mask_binary'] / total_processed
        }
        
        if stats['spill_areas']:
            stats['avg_spill_area'] = np.mean(stats['spill_areas'])
            stats['std_spill_area'] = np.std(stats['spill_areas'])
    
    # Print results
    print(f"\n✅ PROCESSING RESULTS FOR {split.upper()}:")
    print(f"   Total images: {stats['total_images']}")
    print(f"   Successful: {stats['successful_processing']} ({stats['successful_processing']/stats['total_images']*100:.1f}%)")
    print(f"   Failed: {stats['failed_processing']} ({stats['failed_processing']/stats['total_images']*100:.1f}%)")
    
    if total_processed > 0:
        print(f"\n📊 CONSISTENCY VALIDATION:")
        print(f"   Size consistency: {stats['consistency_rate']['size']*100:.1f}%")
        print(f"   Normalization consistency: {stats['consistency_rate']['normalization']*100:.1f}%")
        print(f"   Mask binary consistency: {stats['consistency_rate']['mask_binary']*100:.1f}%")
        
        print(f"\n🎯 CLASS DISTRIBUTION:")
        print(f"   Spill images: {stats['spill_images']} ({stats['spill_ratio']*100:.1f}%)")
        print(f"   Non-spill images: {stats['non_spill_images']} ({(1-stats['spill_ratio'])*100:.1f}%)")
        
        if stats['spill_areas']:
            print(f"   Avg spill area: {stats['avg_spill_area']:.2f}% ± {stats['std_spill_area']:.2f}%")
    
    if stats['processing_errors']:
        print(f"\n❌ PROCESSING ERRORS ({len(stats['processing_errors'])})")
        for error in stats['processing_errors'][:5]:  # Show first 5 errors
            print(f"   {error}")
        if len(stats['processing_errors']) > 5:
            print(f"   ... and {len(stats['processing_errors'])-5} more errors")
    
    return stats

## 7. Complete Pipeline Execution

Execute the complete preprocessing pipeline with all consistency checks and visual verification.

In [ ]:
# Execute complete preprocessing pipeline
print("🚀 STARTING COMPLETE PREPROCESSING PIPELINE")
print("=" * 60)

# Step 1: Process all splits with consistency validation
all_stats = {}
for split in ['train', 'val', 'test']:
    all_stats[split] = process_dataset_split_enhanced(split)

# Step 2: Visual verification for each split
print("\n📸 VISUAL VERIFICATION")
print("=" * 60)
for split in ['train', 'val', 'test']:
    if all_stats[split] is not None:
        print(f"\nVerifying {split} split samples...")
        verify_preprocessing_samples(split, num_samples=3)

# Step 3: Final summary
print("\n🎯 FINAL PREPROCESSING SUMMARY")
print("=" * 60)
for split, stats in all_stats.items():
    if stats is not None:
        print(f"\n{split.upper()} Split:")
        print(f"  ✅ Processed: {stats['successful_processing']}/{stats['total_images']}")
        print(f"  📏 Size consistency: {stats['consistency_rate']['size']*100:.1f}%")
        print(f"  🔢 Normalization consistency: {stats['consistency_rate']['normalization']*100:.1f}%")
        print(f"  🎭 Mask binary consistency: {stats['consistency_rate']['mask_binary']*100:.1f}%")
        print(f"  ⚖️ Spill ratio: {stats['spill_ratio']*100:.1f}%")
        print(f"  🔀 Shuffle setting: {CONFIG[f'shuffle_{split}']}")

print("\n✅ PREPROCESSING PIPELINE COMPLETED SUCCESSFULLY!")
print("\n📋 MILESTONE 1 CHECKLIST:")
print("  ✅ Dataset loaded and explored")
print("  ✅ EDA analysis completed")
print("  ✅ RGB masks converted to binary")
print("  ✅ Images resized to consistent dimensions")
print("  ✅ Normalization applied consistently")
print("  ✅ Augmentation pipeline ready (train only)")
print("  ✅ Dataset balance analyzed")
print("  ✅ Visual verification completed")
print("  ✅ Data ready for model training")

print("\n🚀 Ready to proceed to Milestone 2: Model Training!")